Rand Agent Table with RNN for Time Series:

MODEL 4


* For agents with type Car only
* For all scenes 
* 20 most frequent agents only
* zero-padded (front padding included)

Look at this for more info on multiple parallel series with RNN: https://machinelearningmastery.com/how-to-develop-lstm-models-for-time-series-forecasting/

In [ ]:
import keras
import tensorflow as tf

In [ ]:
from tensorflow.keras.layers import Dense
from tensorflow.keras import Sequential
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import SimpleRNN
from tensorflow.keras.layers import Dropout

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
folderpath = "*"
folderpath = "C:\\Users\\Benson\\Desktop"
file = folderpath + "\\lyftlong\\rand_agents_table0.csv"
at = pd.read_csv(file)
file = folderpath + "\\lyftlong\\rand_frames_table1.csv"
ft = pd.read_csv(file)
file = folderpath +  "\\lyftlong\\rand_scenes_table1.csv"
st = pd.read_csv(file)

In [ ]:
at = at.merge(ft[['frame_index', 'scene_index']], on='frame_index')

In [ ]:
at = at[at.PERCEPTION_LABEL_CAR == 1] 
# FEATURE: filter to only types with Car; uncomment the above to include all agent types

In [ ]:
atcols = ['centroid_x', 'centroid_y', 'track_id', 'frame_index', 'scene_index']

In [ ]:
atrnn = at[atcols]

In [ ]:
scene_indices = pd.unique(at['scene_index'])

In [ ]:
atrnn1 = atrnn[atrnn.scene_index == scene_indices[10]]

In [ ]:
atrnn1s

In [ ]:
agent_parallel_input_series = pd.DataFrame()
apis = agent_parallel_input_series
# apis will be the dataframe that we will partiton for the rnn data table information

atrnn1s = atrnn1 #.sort_values(['track_id', 'frame_index'])

track_ids = pd.unique(atrnn1s['track_id'])
NUM_ENTITIES = len(track_ids)

In [ ]:
# FEATURE: We will only take the 20 most frequent track ids. Set N_MOST_FREQUENT to NUM_ENTITIES if you don't want that

freq_track_ids = atrnn1s.groupby(['track_id']).count().reset_index()

N_MOST_FREQUENT = 20 # NUM_ENTITIES
assert N_MOST_FREQUENT <= NUM_ENTITIES
def get_n_most_frequent(n: int = NUM_ENTITIES):
    """ get's track ids of the n most frequent entities in a scene."""
    global freq_track_ids
    freq_track_ids = freq_track_ids.sort_values('frame_index', ascending=False)
    return freq_track_ids['track_id'].iloc[:20].values
track_ids = get_n_most_frequent(N_MOST_FREQUENT)

In [ ]:
apis_cols = ['centroid_x', 'centroid_y'] + ['frame_index'] #  ['velocity_x', 'velocity_y'] +  ['frame_index'] + ['area']

In [ ]:
apis_frame_indices = pd.unique(atrnn1['frame_index'])

In [ ]:
apis['frame_index'] = apis_frame_indices # added to the apis table so that each agent can be merged 

In [ ]:
for i in range(len(track_ids)):
    if i%100 == 0:
        print("{0} of {1}".format(i, len(track_ids)))
    ti = track_ids[i]
    # for each track id, get it's subset in atrnn1s
    atrnn1sti = atrnn1s[atrnn1s.track_id == ti]
    # get these columns only
    atrnn1sti = atrnn1sti[apis_cols]
    # rename the columns by labelling it with it's track id
    atrnn1sti = atrnn1sti.add_prefix('ti{0}_'.format(str(ti)))
    # undo apis index's renaming for joining to apis
    atrnn1sti.rename(columns = {"ti{0}_frame_index".format(str(ti)): "frame_index"},  
                     inplace = True) 
    # LEFT OUTER JOIN with APIS, atrnn1sti
    apis = pd.merge(apis, atrnn1sti, on='frame_index', how='left')

In [ ]:
apis = apis.drop(['frame_index'], axis=1)

In [ ]:
apis
# if you set N_MOST_FREQUENT to 20, there should be 40 columns only

In [ ]:
# FEATURE: replace all NaN's with zeros
apis = apis.fillna(0)

In [ ]:
atrnn1 = apis

In [ ]:
def make_partitions(ar: np.array, step: int = 3) -> np.array:
    len_ar = len(ar)
    partitions = np.array([])
    for i in range(len_ar - step):
        X_partition = ar[i: i + step]
        y_partition = ar[i + step]
        partition = dict()
        partition['X'] = ((X_partition))
        partition['y'] = ((y_partition))
        partitions = np.append(partitions, partition)
    return partitions

In [ ]:
NSTEPS = 3

In [ ]:
partitions = make_partitions(np.array(atrnn1), NSTEPS)

In [ ]:
TRAIN_PROPORTION = 0.8
TRAIN_PROPORTION_INDEXER = int(len(atrnn1) * 0.8)

In [ ]:
X_train = []
y_train = []
X_test = []
y_test = []
for i in range(len(partitions)): # [:TRAIN_PROPORTION_INDEXER]:
    partition = partitions[i]
    if i <= TRAIN_PROPORTION_INDEXER:
        X_train.append(partition['X'])
        y_train.append(partition['y'])
    else:
        X_test.append(partition['X'])
        y_test.append(partition['y'])        
X_train = np.array(X_train)
y_train = np.array(y_train)
X_test = np.array(X_test)
y_test = np.array(y_test)

In [ ]:
n_steps = NSTEPS
n_features = len(atrnn1.columns)

act = 'tanh'
act1 = 'relu'
recact = 'sigmoid'
retseq = True

#### define model
model = Sequential()
model.add(LSTM(100, activation=act, recurrent_activation=recact, return_sequences=retseq, input_shape=(n_steps, n_features)))
model.add(LSTM(100, activation=act))
model.add(Dense(n_features))
model.compile(optimizer='adam', loss='mse')

model.fit(X_train, y_train, epochs=300, verbose=0)

In [ ]:
model = Sequential()
model.add(LSTM(100,input_shape=(X_train.shape[1], X_train.shape[2]), return_sequences = True))
model.add(Dropout(0.2))
model.add(LSTM(50, activation = 'tanh', return_sequences = True))
model.add(Dropout(0.2))
model.add(LSTM(25, activation = 'tanh', return_sequences = False))
model.add(Dropout(0.2))
model.add(Dense(y_train.shape[1]))
model.compile(loss='mse', optimizer='adam', metrics = ['accuracy'])

model_info = model.fit(X_train, y_train, epochs=300, verbose=0)
print(model_info.loss)

In [ ]:
x_inp = X_test[0]
y_targ = y_test[0]

In [ ]:
x_inp = X_test[0]
y_targ = y_test[0]

x_inp = x_inp = x_inp.reshape((1, n_steps, n_features))
x_inp.shape

y_hat = model.predict(x_inp, verbose=0)[0]

In [ ]:
print(y_targ)

In [ ]:
print(y_hat)

In [ ]:
model_info = model.evaluate(X_test, y_test, batch_size=100)

## SECTION LIMIT

In [ ]:
y_hat = model.predict(x_inp, verbose=0)[0]

In [ ]:
y_hat.astype(int)

In [ ]:
8.4155383e+02, -1.4250477e+03

In [ ]:
y_targ, 

In [ ]:
(1/len(y_hat))*np.sqrt(((y_hat - y_targ)**2)).sum()

In [ ]:
from scipy import spatial

CosineSimilarity = 1 - spatial.distance.cosine(y_hat, y_targ)
CosineSimilarity

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
plt.plot(y_targ)

In [ ]:
plt.plot(y_hat)